In [0]:
%run ../Includes/Classroom-Setup-04.1

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:

CREATE OR REFRESH STREAMING TABLE orders_bronze
AS SELECT current_timestamp() processing_time, input_file_name() source_file, *
FROM cloud_files("${DA.paths.stream_source}/orders", "json", map("cloudFiles.inferColumnTypes", "true"))

Name,Type
processing_time,timestamp
source_file,string
customer_id,bigint
notifications,string
order_id,bigint
order_timestamp,bigint
_rescued_data,string


Resetting the learning environment (pipeline_demo):
| removing the working directory "dbfs:/mnt/dbacademy-users/vivekgaur838@gmail.com/data-engineering-with-databricks/pipeline_demo"...(0 seconds)

Skipping install of existing datasets to "dbfs:/mnt/dbacademy-datasets/data-engineer-learning-path/v04"

Validating the locally installed datasets:
| listing local files...(1 seconds)
| validation completed...(1 seconds total)

Creating & using the schema "vivekgaur838_9msq_da_dewd_pipeline_demo" in the catalog "hive_metastore"...(1 seconds)
Loading batch 1 of 31...0 seconds

Predefined tables in "vivekgaur838_9msq_da_dewd_pipeline_demo":
| -none-

Predefined paths variables:
| DA.paths.working_dir:      dbfs:/mnt/dbacademy-users/vivekgaur838@gmail.com/data-engineering-with-databricks/pipeline_demo
| DA.paths.user_db:          dbfs:/mnt/dbacademy-users/vivekgaur838@gmail.com/data-engineering-with-databricks/pipeline_demo/database.db
| DA.paths.datasets:         dbfs:/mnt/dbacademy-datasets/d

In [0]:
CREATE OR REFRESH STREAMING TABLE orders_silver
(CONSTRAINT valid_date EXPECT (order_timestamp > "2021-01-01") ON VIOLATION FAIL UPDATE)
COMMENT "Append only orders with valid timestamps"
TBLPROPERTIES ("quality" = "silver")
AS SELECT timestamp(order_timestamp) AS order_timestamp, * EXCEPT (order_timestamp, source_file, _rescued_data)
FROM STREAM(LIVE.orders_bronze)

Name,Type
order_timestamp,timestamp
processing_time,timestamp
customer_id,bigint
notifications,string
order_id,bigint


In [0]:
CREATE OR REFRESH LIVE TABLE orders_by_date
AS SELECT date(order_timestamp) AS order_date, count(*) AS total_daily_orders
FROM LIVE.orders_silver
GROUP BY date(order_timestamp)

Name,Type
order_date,date
total_daily_orders,bigint
